# ITER Window test-bed T-Resonator

In [242]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import skrf as rf
from skrf.media import Coaxial

# Importing the DUT S-parameters

In [ ]:
DUT0=rf.Network('../S-parameters/Tresonator_window_front_window.s2p')
DUT0.plot_s_db(m=0, n=0)

# Building the T-resonator

In [380]:
def tres(L_CEA=100, L_DUT=200, f=None):
    Z_short_DUT = 1e-5
    Z_short_CEA = 2e-2

    DUT = rf.Network('../S-parameters/Tresonator_window_front_window.s2p')

    DUT.z0=30
    # the frequency range is the one of the DUT
    if f:
        freq = rf.Frequency(f, f, npoints=1, unit='MHz')
    else:    
        freq = rf.Frequency(start=50, stop=60, npoints=200, unit='MHz')
    DUT = DUT.interpolate(freq)
       
    # electrical conductivities
    conductivity_SS = 1/69e-8 # 1.45e6 in Wikipedia
    conductivity_Cu = 1/1.682e-8 # 5.8e7 # Annealed copper.  5.96e7 for pure Cu in Wikipedia
    conductivity_Cu = 34578645  # Copper 200deg 
    conductivity_Ag = 1/1.59e-8 #6.3e7 # Wikipedia
    conductivity_Al = 1/2.65e-8 #3.8e7 # Wikipedia

    Tee = Coaxial(frequency=freq, Dint=0.140,  Dout=0.230, epsilon_r=1, sigma=conductivity_Ag).tee(name='Tee')

    # CEA Branch
    C0 = Coaxial(frequency=freq, Dint=0.140,  Dout=0.230, epsilon_r=1, sigma=conductivity_Ag).line(200, unit='mm', name='C0')  # T (1/2)
    C1 = Coaxial(frequency=freq, Dint=0.130,  Dout=0.294, epsilon_r=1, sigma=conductivity_Ag).line(180, unit='mm', name='C1')
    C2 = Coaxial(frequency=freq, Dint=0.140,  Dout=0.230, epsilon_r=1, sigma=conductivity_Al).line(2000, unit='mm', name='C2')
    _C3 = Coaxial(frequency=freq, Dint=0.140,  Dout=0.230, epsilon_r=1, sigma=conductivity_Al)
    C3 = _C3.line(L_CEA, unit='mm', name='C3')

    # DUT Branch
    D0 = Coaxial(frequency=freq, Dint=0.140,  Dout=0.230, epsilon_r=1, sigma=conductivity_Ag).line(200, unit='mm', name='D0') # T (1/2)
    D1 = Coaxial(frequency=freq, Dint=0.140, Dout=0.230, epsilon_r=1, sigma=conductivity_Ag).line(100, unit='mm', name='D1')
    LB1 = Coaxial(frequency=freq, Dint=0.140, Dout=0.410, epsilon_r=1, sigma=conductivity_Cu).line(239, unit='mm', name='LB1')
    LB2 = Coaxial(frequency=freq, Dint=0.140, Dout=0.230, epsilon_r=1, sigma=conductivity_Cu).line(1224, unit='mm', name='LB2')
    LB3 = Coaxial(frequency=freq, Dint=0.140, Dout=0.230, epsilon_r=1, sigma=conductivity_Cu).line(200, unit='mm', name='LB3')
    _LB5 = Coaxial(frequency=freq, Dint=0.140, Dout=0.230, epsilon_r=1, sigma=conductivity_Cu)
    LB5 = _LB5.line(L_DUT, unit='mm', name='LB5')

    # T-Resonator Input port
    port1 = rf.Circuit.Port(frequency=freq, z0=30, name='port1')
    # Short circuits
    resistor_dut = _LB5.resistor(Z_short_DUT, name='short_dut')
    resistor_cea = _C3.resistor(Z_short_CEA, name='short_cea')
    gnd_dut = rf.Circuit.Ground(frequency=freq, z0=_LB5.z0[0], name='gnd_dut')
    gnd_cea = rf.Circuit.Ground(frequency=freq, z0=_C2.z0[0], name='gnd_cea')

    cnx = [
        # T-junction
        [(port1, 0), (Tee, 0)], 
        [(Tee, 1), (C0, 0)], [(Tee, 2), (D0, 0)],
        # DUT Branch
        [(D0, 1), (D1, 0)],
        [(D1, 1), (LB1, 0)],
        [(LB1, 1), (LB2, 0)],
        [(LB2, 1), (LB3, 0)],
        [(LB3, 1), (DUT, 1)],
        [(DUT, 0), (LB5, 0)],
        [(LB5, 1), (resistor_dut, 0)],
        [(resistor_dut, 1), (gnd_dut, 0)],
        # CEA branch
        [(C0, 1), (C1, 0)],
        [(C1, 1), (C2, 0)],
        [(C2, 1), (C3, 0)],
        [(C3, 1), (resistor_cea, 0)],
        [(resistor_cea, 1), (gnd_cea, 0)],
    ]
    circuit = rf.Circuit(cnx)
    return circuit.network

def s11_tresonator(L_CEA=100, L_DUT=200):
    fig, ax = plt.subplots()
    tres(L_CEA, L_DUT).plot_s_db(ax=ax)
    ax.set_ylim(-30,1)


In [381]:
interact(s11_tresonator, 
         L_CEA=widgets.FloatSlider(min=1, max=500, step=1, value=100), 
         L_DUT=widgets.FloatSlider(min=1, max=500, step=1, value=200))

interactive(children=(FloatSlider(value=100.0, description='L_CEA', max=500.0, min=1.0, step=1.0), FloatSlider…

<function __main__.s11_tresonator(L_CEA=100, L_DUT=200)>

In [382]:
L_CEAs = np.linspace(0, 300, 50)
fs = np.linspace(50, 60, 50)

LL_CEA, LL_fs = np.meshgrid(L_CEAs, fs)

In [383]:
def S11_f_match(d_CEA, f):
    _res = tres(L_CEA=d_CEA, f=f)
    
    return np.squeeze(_res.s_db[0])

vfunc = np.vectorize(S11_f_match)

In [ ]:
S11 = vfunc(LL_CEA, LL_fs)

In [ ]:
fig,ax=plt.subplots()
c=ax.pcolor(LL_CEA, fs, S11, cmap='Greens_r')
fig.colorbar(c)
# ax.axvline(d_DUT_opt*1e3, ls='--')
# ax.axhline(d_CEA_opt*1e3, ls='--')

ax.set_xlabel('L CEA [mm]')
ax.set_ylabel('f [MHz]')
ax.set_title('s11 [dB]')